In [27]:
from sklearn.datasets import load_iris
import pandas as pd
from pandasql import sqldf
from pandasql import load_meat, load_births
from IPython.display import display
import re

### Load Source Data

In [36]:
births = load_births() # DataFrame
meat = load_meat()    # DataFrame
iris = load_iris()    # sklearn.datasets.base.Bunch
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names) # params: ndarray, list
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names) # Make a Categorical type from codes and categories arrays.
iris_df.columns = [re.sub("[() ]", "", col) for col in iris_df.columns]  # remove chars

display(type(iris_df))
display(iris_df.head())
display(len(iris_df))

pandas.core.frame.DataFrame

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


150

### Basic SELECT

In [34]:
# locals() Returns a dictionary containing the current scope's local variables.
display(sqldf("SELECT * FROM iris_df LIMIT 10;", locals()))
display(sqldf("SELECT sepalwidthcm, species FROM iris_df LIMIT 10;", locals()))

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


,sepalwidthcm,species
0,3.5,setosa
1,3.0,setosa
2,3.2,setosa
3,3.1,setosa
4,3.6,setosa
5,3.9,setosa
6,3.4,setosa
7,3.4,setosa
8,2.9,setosa
9,3.1,setosa


### Aggregation

In [40]:
q = """
      SELECT
        species
        , AVG(sepalwidthcm)
        , MIN(sepalwidthcm)
        , MAX(sepalwidthcm)
      FROM
        iris_df
      GROUP BY
        species; 
"""
sqldf(q, locals())

,species,AVG(sepalwidthcm),MIN(sepalwidthcm),MAX(sepalwidthcm)
0,setosa,3.418,2.3,4.4
1,versicolor,2.770,2.0,3.4
2,virginica,2.974,2.2,3.8


### JOIN 

In [46]:
q = """
    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            ON m.date = b.date
    ORDER BY
        m.date
    LIMIT 10;
"""
sqldf(q, locals())

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey,births
0,1975-01-01 00:00:00.000000,2106.0,59.0,1114.0,36.0,646.2,None,64.9,265775
1,1975-02-01 00:00:00.000000,1845.0,50.0,954.0,31.0,570.2,None,47.1,241045
2,1975-03-01 00:00:00.000000,1891.0,57.0,976.0,35.0,616.6,None,54.4,268849
3,1975-04-01 00:00:00.000000,1895.0,60.0,1100.0,34.0,688.3,None,68.7,247455
4,1975-05-01 00:00:00.000000,1849.0,59.0,934.0,31.0,690.1,None,81.9,254545
5,1975-06-01 00:00:00.000000,1849.0,63.0,889.0,31.0,683.1,None,138.4,254096
6,1975-07-01 00:00:00.000000,1916.0,77.0,817.0,32.0,714.2,None,193.2,275163
7,1975-08-01 00:00:00.000000,1961.0,73.0,794.0,32.0,680.5,None,203.3,281300
8,1975-09-01 00:00:00.000000,2065.0,82.0,901.0,40.0,684.9,None,229.0,270738
9,1975-10-01 00:00:00.000000,2270.0,95.0,936.0,38.0,739.8,None,257.5,265494


### WHERE

In [42]:
q = """
    SELECT
        *
    FROM
        iris_df
    WHERE
        species = 'virginica'
        and sepallengthcm > 7.7;
"""
sqldf(q, locals())

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,7.9,3.8,6.4,2.0,virginica


### SUBQUERY

In [47]:
iris_df['id'] = range(len(iris_df))
q = """
    SELECT
        *
    FROM
        iris_df
    WHERE
        id IN (SELECT id FROM iris_df WHERE sepalwidthcm*sepallengthcm > 25);
"""
sqldf(q, locals())

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species,id
0,5.7,4.4,1.5,0.4,setosa,15
1,7.2,3.6,6.1,2.5,virginica,109
2,7.7,3.8,6.7,2.2,virginica,117
3,7.9,3.8,6.4,2.0,virginica,131
